# Import Libraries

In [54]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Import cleaned_listings.pkl (refer to Airbnb Data Pre-processing notebook #2 for cleaned_listings.pkl)

In [55]:
SF_df = pd.read_pickle("cleaned_listings.pkl")
SF_df.head(5)

,id,name,description,neighborhood_overview,host_id,host_name,host_since,host_location,host_about,host_response_time,...,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,reviews_per_month
0,958,"Bright, Modern Garden Unit - 1BR/1BTH",Cleaning Protocol: Our cleaners follow the Air...,Quiet cul de sac in friendly neighborhood<br /...,1169,Holly,2008-07-31,"San Francisco, California, United States",We are a family of four that live upstairs. W...,within an hour,...,2020-07-30,97.0,10.0,10.0,10.0,10.0,10.0,10.0,False,1.91
1,5858,Creative Sanctuary,<b>The space</b><br />We live in a large Victo...,I love how our neighborhood feels quiet but is...,8904,Philip And Tania,2009-03-02,"San Francisco, California, United States",Philip: English transplant to the Bay Area and...,within a day,...,2017-08-06,98.0,10.0,10.0,10.0,10.0,10.0,9.0,False,0.81
2,7918,A Friendly Room - UCSF/USF - San Francisco,Nice and good public transportation. 7 minute...,"Shopping old town, restaurants, McDonald, Whol...",21994,Aaron,2009-06-17,"San Francisco, California, United States",7 minutes walk to UCSF hospital & school campu...,within a few hours,...,2020-03-06,84.0,7.0,8.0,9.0,9.0,9.0,8.0,False,0.14
3,8142,Friendly Room Apt. Style -UCSF/USF - San Franc...,Nice and good public transportation. 7 minute...,NaN,21994,Aaron,2009-06-17,"San Francisco, California, United States",7 minutes walk to UCSF hospital & school campu...,within a few hours,...,2018-09-12,93.0,9.0,9.0,10.0,10.0,9.0,9.0,False,0.11
4,8339,Historic Alamo Square Victorian,Pls email before booking. <br />Interior featu...,NaN,24215,Rosy,2009-07-02,"San Francisco, California, United States",I'm an Interior Stylist living in SF. \r\n\r\n...,within a few hours,...,2019-06-28,97.0,10.0,10.0,10.0,10.0,10.0,10.0,False,0.21


In [56]:
SF_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7274 entries, 0 to 7273
Data columns (total 56 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   id                           7274 non-null   int64         
 1   name                         7274 non-null   object        
 2   description                  7220 non-null   object        
 3   neighborhood_overview        5330 non-null   object        
 4   host_id                      7274 non-null   int64         
 5   host_name                    7274 non-null   object        
 6   host_since                   7274 non-null   datetime64[ns]
 7   host_location                7269 non-null   object        
 8   host_about                   5475 non-null   object        
 9   host_response_time           5321 non-null   object        
 10  host_response_rate           5321 non-null   float64       
 11  host_acceptance_rate         6393 non-null 

# Drop Columns Based on Pearson Correlation (refer to Price Analysis notebook)

In [57]:
SF_df = SF_df.drop(columns=['bedrooms','beds','maximum_nights','number_of_reviews'])

# Create lists for different data type columns

In [58]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numeric_columns = SF_df.select_dtypes(include=numerics).columns.to_list()
bool_columns = SF_df.select_dtypes(include='bool').columns.to_list()
object_columns = SF_df.select_dtypes(include='object').columns.to_list()
print(object_columns)

['name', 'description', 'neighborhood_overview', 'host_name', 'host_location', 'host_about', 'host_response_time', 'host_neighbourhood', 'host_verifications', 'neighbourhood', 'neighbourhood_cleansed', 'property_type', 'room_type', 'bathrooms_text', 'amenities']


# Convert boolean columns to numeric columns

In [59]:
for column in bool_columns:
    SF_df[column] = SF_df[column].replace(False,0,regex=True)
    SF_df[column] = SF_df[column].replace(True,1,regex=True)
SF_df["host_is_superhost"].head()

0    1.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: host_is_superhost, dtype: float64

# Transforming amenities to numeric columns

In [60]:
SF_df.amenities = SF_df.amenities.str.replace('"', "")
SF_df.amenities = SF_df.amenities.str.replace("[", "")
SF_df.amenities = SF_df.amenities.str.replace("]", "")
SF_df['amenities'].head()

0    Heating, Hot water, Stove, Iron, Dryer, Coffee...
1    Smoke alarm, Heating, Kitchen, First aid kit, ...
2    Host greets you, Heating, Hot water, Kitchen, ...
3    Host greets you, Heating, Hot water, Kitchen, ...
4    Heating, Hot water, Stove, Iron, Carbon monoxi...
Name: amenities, dtype: object

In [61]:
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer =  CountVectorizer(tokenizer=lambda x: x.split(', '))
amenities = count_vectorizer.fit_transform(SF_df['amenities'])
df_amenities = pd.DataFrame(amenities.toarray(), columns=count_vectorizer.get_feature_names())
df_amenities = df_amenities.drop('',1)

# Combining numeric and boolean columns

In [62]:
listings_new = SF_df[numeric_columns+bool_columns]
listings_new.head()

,id,host_id,host_response_rate,host_acceptance_rate,host_listings_count,host_total_listings_count,latitude,longitude,accommodates,bathrooms,...,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month,host_is_superhost,host_has_profile_pic,host_identity_verified,has_availability,instant_bookable
0,958,1169,1.0,0.99,1,1,37.76931,-122.43386,3,1.0,...,10.0,10.0,10.0,10.0,1.91,1.0,1.0,1.0,1.0,0.0
1,5858,8904,0.6,0.83,2,2,37.74511,-122.42102,5,1.0,...,10.0,10.0,10.0,9.0,0.81,0.0,1.0,1.0,1.0,0.0
2,7918,21994,1.0,1.00,10,10,37.76555,-122.45213,2,4.0,...,9.0,9.0,9.0,8.0,0.14,0.0,1.0,0.0,1.0,0.0
3,8142,21994,1.0,1.00,10,10,37.76555,-122.45213,2,4.0,...,10.0,10.0,9.0,9.0,0.11,0.0,1.0,0.0,1.0,0.0
4,8339,24215,1.0,0.00,2,2,37.77525,-122.43637,4,1.5,...,10.0,10.0,10.0,10.0,0.21,0.0,1.0,1.0,1.0,0.0


# Transforming categorical columns to numeric columns and combine all into new df

In [63]:
for categorical_col in object_columns[:-1]:
    listings_new = pd.concat([listings_new, pd.get_dummies(SF_df[categorical_col])], axis=1)

# Combine all that with amenities df created in #4

In [64]:
listings_new = pd.concat([listings_new, df_amenities], axis=1, join='inner')
listings_new.head()

,id,host_id,host_response_rate,host_acceptance_rate,host_listings_count,host_total_listings_count,latitude,longitude,accommodates,bathrooms,...,terrace,tv,walk in closet,washer,waterfront,wet bar,wifi,window guards,wine cooler,wood-burning fireplace
0,958,1169,1.0,0.99,1,1,37.76931,-122.43386,3,1.0,...,0,1,0,1,0,0,1,0,0,0
1,5858,8904,0.6,0.83,2,2,37.74511,-122.42102,5,1.0,...,0,0,0,1,0,0,1,0,0,0
2,7918,21994,1.0,1.00,10,10,37.76555,-122.45213,2,4.0,...,0,1,0,1,0,0,1,0,0,0
3,8142,21994,1.0,1.00,10,10,37.76555,-122.45213,2,4.0,...,0,1,0,1,0,0,1,0,0,0
4,8339,24215,1.0,0.00,2,2,37.77525,-122.43637,4,1.5,...,0,1,0,0,0,0,1,0,0,0


In [65]:
listings_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7274 entries, 0 to 7273
Columns: 23096 entries, id to wood-burning fireplace
dtypes: float64(21), int64(143), uint8(22932)
memory usage: 168.2 MB


# Check and drop duplicates if any

In [ ]:
print(listings_new.duplicated().any())
if listings_new.duplicated().any():
    duplicate_rows = listings_new[listings_new.duplicated()]
    listings_new = listings_new.drop_duplicates()
    print("Duplicates Removed")
print(listings_new.duplicated().any())

False


# Drop the problematic rows with 68 dollar price

In [ ]:
listings_new[listings_new['price'] == 68].shape

In [ ]:
listings_new = listings_new[listings_new['price'] != 68]
listings_new.info()

# Rename duplicate column names

In [ ]:
# rename duplicate column
listings_new.columns = \
['host_response_rate',
 'host_acceptance_rate',
 'host_is_superhost',
 'host_listings_count',
 'accommodates',
 'bathrooms',
 'price',
 'minimum_nights',
 'review_scores_rating',
 'reviews_per_month',
 'price_log',
 '-',
 'a few days or more',
 'within a day',
 'within a few hours',
 'within an hour',
 'Bayview',
 'Bernal Heights',
 'Castro/Upper Market',
 'Chinatown',
 'Crocker Amazon',
 'Diamond Heights',
 'Downtown/Civic Center',
 'Excelsior',
 'Financial District',
 'Glen Park',
 'Golden Gate Park',
 'Haight Ashbury',
 'Inner Richmond',
 'Inner Sunset',
 'Lakeshore',
 'Marina',
 'Mission',
 'Nob Hill',
 'Noe Valley',
 'North Beach',
 'Ocean View',
 'Outer Mission',
 'Outer Richmond',
 'Outer Sunset',
 'Pacific Heights',
 'Parkside',
 'Potrero Hill',
 'Presidio',
 'Presidio Heights',
 'Russian Hill',
 'Seacliff',
 'South of Market',
 'Treasure Island/YBI',
 'Twin Peaks',
 'Visitacion Valley',
 'West of Twin Peaks',
 'Western Addition',
 'Boat',
 'Casa particular',
 'Earth house',
 'Entire apartment',
 'Entire bungalow',
 'Entire cabin',
 'Entire condominium',
 'Entire cottage',
 'Entire floor',
 'Entire guest suite',
 'Entire guesthouse',
 'Entire house',
 'Entire in-law',
 'Entire loft',
 'Entire place',
 'Entire serviced apartment',
 'Entire townhouse',
 'Entire villa',
 'Private room (Property Type)',
 'Private room in apartment',
 'Private room in bed and breakfast',
 'Private room in bungalow',
 'Private room in cabin',
 'Private room in castle',
 'Private room in condominium',
 'Private room in cottage',
 'Private room in earth house',
 'Private room in guest suite',
 'Private room in guesthouse',
 'Private room in hostel',
 'Private room in house',
 'Private room in hut',
 'Private room in loft',
 'Private room in resort',
 'Private room in serviced apartment',
 'Private room in townhouse',
 'Private room in villa',
 'Room in aparthotel',
 'Room in bed and breakfast',
 'Room in boutique hotel',
 'Room in hostel',
 'Room in hotel',
 'Room in serviced apartment',
 'Shared room in apartment',
 'Shared room in bed and breakfast',
 'Shared room in boutique hotel',
 'Shared room in bungalow',
 'Shared room in condominium',
 'Shared room in hostel',
 'Shared room in house',
 'Shared room in loft',
 'Shared room in townhouse',
 'Shared room in villa',
 'Tiny house',
 'Entire home/apt',
 'Hotel room',
 'Private room',
 'Shared room',
 'air conditioning',
 'alarm system',
 'baby bath',
 'baby monitor',
 'babysitter recommendations',
 'baking sheet',
 'barbecue utensils',
 'bathroom essentials',
 'bathtub',
 'bbq grill',
 'beach essentials',
 'beachfront',
 'bed linens',
 'bedroom comforts',
 'bread maker',
 'breakfast',
 'breakfast bar',
 'building staff',
 'cable tv',
 'carbon monoxide alarm',
 'changing table',
 'children\\u2019s books and toys',
 'children\\u2019s dinnerware',
 'cleaning before checkout',
 'coffee maker',
 'cooking basics',
 'crib',
 'desk',
 'dishes and silverware',
 'dishwasher',
 'dryer',
 'dual vanity',
 'elevator',
 'essentials',
 'ethernet connection',
 'ev charger',
 'extra pillows and blankets',
 'fire extinguisher',
 'fireplace guards',
 'first aid kit',
 'free parking on premises',
 'free street parking',
 'full kitchen',
 'game console',
 'garden or backyard',
 'gas fireplace',
 'gym',
 'hair dryer',
 'hangers',
 'heating',
 'high chair',
 'host greets you',
 'hot tub',
 'hot water',
 'indoor fireplace',
 'iron',
 'ironing board',
 'jetted tub',
 'keypad',
 'kitchen',
 'kitchenette',
 'lake access',
 'laptop-friendly workspace',
 'lock on bedroom door',
 'lockbox',
 'long term stays allowed',
 'luggage dropoff allowed',
 'microwave',
 'mini fridge',
 'natural gas barbeque',
 'office',
 'outlet covers',
 'oven',
 'pack \\u2019n play/travel crib',
 'paid parking off premises',
 'paid parking on premises',
 'patio or balcony',
 'pets allowed',
 'piano',
 'pocket wifi',
 'pool',
 'portable air conditioning',
 'private entrance',
 'private hot tub',
 'private living room',
 'refrigerator',
 'room-darkening shades',
 'security cameras',
 'self check-in',
 'shampoo',
 'shared hot tub',
 'shower gel',
 'single level home',
 'ski-in/ski-out',
 'smart home technology',
 'smart lock',
 'smart tv',
 'smoke alarm',
 'smoking allowed',
 'sonos sound system',
 'stair gates',
 'stand alone bathtub',
 'stand alone rain shower',
 'stove',
 'suitable for events',
 'table corner guards',
 'terrace',
 'tv',
 'walk in closet',
 'washer',
 'waterfront',
 'wet bar',
 'wifi',
 'window guards',
 'wine cooler']

# Drop '-' column

In [ ]:
listings_new.drop(columns=['-'],inplace=True)

In [ ]:
listings_new.info()

# Export to pickle

In [ ]:
# listings_new.to_pickle('final_cleaned_df.pkl')


listings_new = pd.read_pickle('final_cleaned_df.pkl')